In [5]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
import torch.optim as optim




ModuleNotFoundError: No module named 'torch'

In [3]:

m=6000
text=[]
summary=[]
df=pd.read_csv('/kaggle/input/arabic-text-summarization-30-000/wikiHow.csv')
for i in range(df.shape[0]):
    if len(df['text'][i])< m:
        text.append(df['text'][i])
        summary.append(df['summary'][i])

df = pd.DataFrame({'text': text, 'summary': summary})
df['summary'] = df['summary'].replace(r'\n', '', regex = True)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/arabic-text-summarization-30-000/wikiHow.csv'

In [3]:

from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.1, random_state=42)

train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch


train_data=pd.read_csv('/kaggle/working/train.csv')
test_data=pd.read_csv('/kaggle/working/val.csv')


# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        summary = self.data.iloc[idx]["summary"]

        inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        labels = self.tokenizer(
            summary,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten(),
            "labels": labels["input_ids"].flatten(),
        }




In [5]:
# Define hyperparameters
MAX_LENGTH = 512
BATCH_SIZE = 6
EPOCHS = 2


# Load BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Prepare dataset and dataloaders
train_dataset = CustomDataset(train_data, tokenizer, MAX_LENGTH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(EPOCHS):
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(train_loader)}")



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Epoch 1/2: 100%|██████████| 4330/4330 [1:41:36<00:00,  1.41s/it]


Epoch 1/2, Loss: 0.4571687105862428


Epoch 2/2: 100%|██████████| 4330/4330 [1:41:20<00:00,  1.40s/it]

Epoch 2/2, Loss: 0.3587297078609191


In [6]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_bart_model")
tokenizer.save_pretrained("fine_tuned_bart_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('fine_tuned_bart_model/tokenizer_config.json',
 'fine_tuned_bart_model/special_tokens_map.json',
 'fine_tuned_bart_model/vocab.json',
 'fine_tuned_bart_model/merges.txt',
 'fine_tuned_bart_model/added_tokens.json')

In [7]:

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Create data module
dm = SummaryDataModule(
    train_path="./train.csv",
    val_path="./val.csv",
    tokenizer=tokenizer,
    batch_size=2,  # Adjust batch size as needed
    text_max_token_len=1000  # Maximum input token length supported by BART
)

# Create BART summarizer model
model = BartSummarizer(tokenizer)

# Define early stopping callback
early_stop_callback = EarlyStopping(monitor='val_loss')

# Train the model
trainer = pl.Trainer(
    max_epochs=3, 
    default_root_dir='.'
)
trainer.fit(model, dm)

NameError: name 'SummaryDataModule' is not defined

In [ ]:
# Load BART tokenizer and model
pre_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
pre_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
pre_model.to(device)
import torch

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the appropriate device
model.to(device)

def summarizeText(text, mymodel):
    # Move input tensors to the same device as the model
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    ).to(device)

    generated_ids = mymodel.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
test = test_data.sample(5)
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    print('Text : ', text)
    print('\n')
    print('Summary (Ground truth) : ',sample_row['summary'])
    print('\n')
    summary_without_tuning = summarizeText(text, pre_model)
    print("Summary before tuning", summary_without_tuning)
    print('\n')
    summary_with_tuning = summarizeText(text, model)
    print("Summary after tuning", summary_with_tuning)
    print('\n')
    print('-------------------------------------------------------------------------------------------------')

In [ ]:
pip install rouge_score


In [ ]:
from rouge_score import rouge_scorer
test=test_data.sample(10)
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual ROUGE scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Iterate through test samples and corresponding summaries
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    summary_with_tuning = summarizeText(text, model)

    # Compute ROUGE scores for each sample
    scores = scorer.score(text, summary_with_tuning)

    # Store individual ROUGE scores
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)